In [1]:
import pandas as pd
import numpy as np

from collections import Counter
from itertools import chain

In [2]:
train = pd.read_csv("sessions_train.txt", names=["look", "buy"], sep=';')
train.look = train.look.str.split(",")
train.buy = train.buy.str.split(",")
train.head()

,look,buy
0,"[0, 1, 2, 3, 4, 5]",NaN
1,"[9, 10, 11, 9, 11, 12, 9, 11]",NaN
2,"[16, 17, 18, 19, 20, 21]",NaN
3,"[24, 25, 26, 27, 24]",NaN
4,"[34, 35, 36, 34, 37, 35, 36, 37, 38, 39, 38, 39]",NaN


In [3]:
test = pd.read_csv("sessions_test.txt", names=["look", "buy"], sep=';')
test.look = test.look.str.split(",")
test.buy = test.buy.str.split(",")
test.dropna(inplace=True)
test.head()

,look,buy
7,"[63, 68, 69, 70, 66, 61, 59, 61, 66, 68]","[66, 63]"
14,"[158, 159, 160, 159, 161, 162]",[162]
19,"[200, 201, 202, 203, 204]","[201, 205]"
34,"[371, 372, 371]","[371, 373]"
40,[422],[422]


На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)

In [4]:
train_look_counter = Counter(chain.from_iterable(train.look.values))
train.dropna(inplace=True)
train_buy_counter = Counter(chain.from_iterable(train.buy.values))

In [5]:
def precision(l, k):
    # how many from recomendations are bought
    return l / k

def recall(l, n):
    # how many recomendations are part of buy
    return l / n
    
def recomend(look, max_k, f):
    look = pd.unique(look)
    k = min(len(look), max_k) #k recommendations to show
    predict = list(look)
    predict.sort(key=f, reverse=True)
    return predict[:k]

In [6]:
train_predict = train.copy()
train_predict["recomend_look_1"] = train.apply(lambda row: recomend(row.look, 1, lambda x: train_look_counter[str(x)]), axis=1)
train_predict["recomend_look_5"] = train.apply(lambda row: recomend(row.look, 5, lambda x: train_look_counter[str(x)]), axis=1)
train_predict["recomend_buy_1"] = train.apply(lambda row: recomend(row.look, 1, lambda x: train_buy_counter[str(x)]), axis=1)
train_predict["recomend_buy_5"] = train.apply(lambda row: recomend(row.look, 5, lambda x: train_buy_counter[str(x)]), axis=1)
train_predict.head(5)

,look,buy,recomend_look_1,recomend_look_5,recomend_buy_1,recomend_buy_5
7,"[59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 6...","[67, 60, 63]",[63],"[63, 64, 60, 61, 65]",[60],"[60, 63, 67, 59, 61]"
10,"[84, 85, 86, 87, 88, 89, 84, 90, 91, 92, 93, 86]",[86],[85],"[85, 93, 89, 90, 84]",[86],"[86, 85, 93, 84, 87]"
19,"[138, 198, 199, 127]",[199],[127],"[127, 138, 198, 199]",[138],"[138, 199, 127, 198]"
30,"[303, 304, 305, 306, 307, 308, 309, 310, 311, ...",[303],[303],"[303, 306, 304, 307, 309]",[303],"[303, 304, 305, 306, 307]"
33,"[352, 353, 352]",[352],[352],"[352, 353]",[352],"[352, 353]"


In [7]:
test_predict = test.copy()
test_predict["recomend_look_1"] = test.apply(lambda row: recomend(row.look, 1, lambda x: train_look_counter[str(x)]), axis=1)
test_predict["recomend_look_5"] = test.apply(lambda row: recomend(row.look, 5, lambda x: train_look_counter[str(x)]), axis=1)
test_predict["recomend_buy_1"] = test.apply(lambda row: recomend(row.look, 1, lambda x: train_buy_counter[str(x)]), axis=1)
test_predict["recomend_buy_5"] = test.apply(lambda row: recomend(row.look, 5, lambda x: train_buy_counter[str(x)]), axis=1)
test_predict.head(5)

,look,buy,recomend_look_1,recomend_look_5,recomend_buy_1,recomend_buy_5
7,"[63, 68, 69, 70, 66, 61, 59, 61, 66, 68]","[66, 63]",[63],"[63, 68, 66, 61, 59]",[63],"[63, 68, 69, 70, 66]"
14,"[158, 159, 160, 159, 161, 162]",[162],[158],"[158, 162, 160, 159, 161]",[158],"[158, 162, 160, 159, 161]"
19,"[200, 201, 202, 203, 204]","[201, 205]",[204],"[204, 202, 203, 200, 201]",[204],"[204, 202, 200, 201, 203]"
34,"[371, 372, 371]","[371, 373]",[371],"[371, 372]",[371],"[371, 372]"
40,[422],[422],[422],[422],[422],[422]


In [20]:
train_predict2 = pd.DataFrame()
for col, k in zip(train_predict.columns[2:],[1,5,1 ,5]):
    train_predict2[col + "_recall"] = train_predict.apply(lambda row: recall(len(set(row.buy).intersection(row[col])), len(row.buy)), axis=1)
    train_predict2[col + "_precision"] = train_predict.apply(lambda row: precision(len(set(row.buy).intersection(row[col])), k), axis=1)

train_predict2.head()

,recomend_look_1_recall,recomend_look_1_precision,recomend_look_5_recall,recomend_look_5_precision,recomend_buy_1_recall,recomend_buy_1_precision,recomend_buy_5_recall,recomend_buy_5_precision
7,0.333333,1.0,0.666667,0.4,0.333333,1.0,1.0,0.6
10,0.000000,0.0,0.000000,0.0,1.000000,1.0,1.0,0.2
19,0.000000,0.0,1.000000,0.2,0.000000,0.0,1.0,0.2
30,1.000000,1.0,1.000000,0.2,1.000000,1.0,1.0,0.2
33,1.000000,1.0,1.000000,0.2,1.000000,1.0,1.0,0.2


In [21]:
test_predict2 = pd.DataFrame()
for col, k in zip(test_predict.columns[2:],[1,5,1,5]):
    test_predict2[col + "_recall"] = test_predict.apply(lambda row: recall(len(set(row.buy).intersection(row[col])), len(row.buy)), axis=1)
    test_predict2[col + "_precision"] = test_predict.apply(lambda row: precision(len(set(row.buy).intersection(row[col])), k), axis=1)

test_predict2.head()

,recomend_look_1_recall,recomend_look_1_precision,recomend_look_5_recall,recomend_look_5_precision,recomend_buy_1_recall,recomend_buy_1_precision,recomend_buy_5_recall,recomend_buy_5_precision
7,0.5,1.0,1.0,0.4,0.5,1.0,1.0,0.4
14,0.0,0.0,1.0,0.2,0.0,0.0,1.0,0.2
19,0.0,0.0,0.5,0.2,0.0,0.0,0.5,0.2
34,0.5,1.0,0.5,0.2,0.5,1.0,0.5,0.2
40,1.0,1.0,1.0,0.2,1.0,1.0,1.0,0.2


In [10]:
# Функция сохранения в файл ответа, представленного массивом
def save_answerArray(fname,array):
    with open(fname,"w") as fout:
        fout.write(" ".join([str(el) for el in array]))

In [14]:
#0.4Х 0.5Х 0.8Х 0.2Х
result1 = []
for col in train_predict2.columns[:4]:
    average = train_predict2[col].mean()
    print(average)
    result1.append(f"{average:.2f}")
    print(f"{average:.2f}")
    
save_answerArray("q1", result1)

0.4426343165949593
0.44
0.5121951219512195
0.51
0.8246918247126122
0.82
0.21252771618625918
0.21


In [19]:
result2 = []
for col in test_predict2.columns[:4]:
    average = test_predict2[col].mean()
    print(average)
    result2.append(f"{average:.2f}")
    print(f"{average:.2f}")
    
save_answerArray("q2", result2)

0.41733266203252534
0.42
0.48130968622100956
0.48
0.8000340663538579
0.80
0.2037653478854079
0.20


In [22]:
#0.4Х 0.5Х 0.8Х 0.2Х
result3 = []
for col in train_predict2.columns[4:]:
    print(col)
    average = train_predict2[col].mean()
    print(average)
    result3.append(f"{average:.2f}")
    print(f"{average:.2f}")
    
save_answerArray("q3", result3)

recomend_buy_1_recall
0.6884494924267653
0.69
recomend_buy_1_precision
0.8037694013303769
0.80
recomend_buy_5_recall
0.9263073024228787
0.93
recomend_buy_5_precision
0.2525498891352649
0.25


In [23]:
result4 = []
for col in test_predict2.columns[4:]:
    average = test_predict2[col].mean()
    print(average)
    result4.append(f"{average:.2f}")
    print(f"{average:.2f}")
    
save_answerArray("q4", result4)

0.4606201666660294
0.46
0.5276944065484311
0.53
0.8201874337490194
0.82
0.21009549795362173
0.21
